<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Chapter


## 顶层设置时钟域设置成你的风格

In [ ]:
class Top extends Component{
    val clk,rstn = in Bool
    
    val io = new Bundle{
      val a = in UInt(8 bits)
      val b = out UInt(8 bits)        
    }
    
    val TopClockDomain = ClockDomain(clk,rstn,config=ClockDomainConfig(resetKind = ASYNC,
        clockEdge = RISING, 
        resetActiveLevel = LOW))
    
val clockarea = new ClockingArea(TopClockDomain){
    val u_sub = new SubModule(8)
    u_sub.io.a := io.a
    io.b := u_sub.io.b
}
}
showRtl(new Top)

In [ ]:
class SUB extends Component{
    val a = in Bits(8 bits)
    val b = out(RegNext(a) init 0)
}
showRtl(new SUB)

In [ ]:
class Top extends Component{
    val myclk,myrst = in Bool()
    val a = in Bits(8 bits)
    val b = out Bits()
    new ClockingArea(ClockDomain(myclk,myrst, 
                                 config = ClockDomainConfig(
      clockEdge        = RISING,
      resetKind        = ASYNC,
      resetActiveLevel = LOW
    ))){
      val reg0 = RegNext(a) init 0 
      b := reg0
    }
    
}
showRtl(new Top)


In [ ]:
class Top extends Component{
    val myclk,myrst = in Bool()
    val a = in Bits(8 bits)
    val b = out Bits(8 bits)
    val cd = ClockDomain(myclk,myrst, 
      config = ClockDomainConfig(
      clockEdge        = RISING,
      resetKind        = ASYNC,
      resetActiveLevel = LOW
    ))
    val u_sub0 = cd(new SUB)
    u_sub0.a := a
    b := u_sub0.b
    
}
showRtl(new Top)

In [19]:
class Top extends Component{
    val myclk,myrst = in Bool()
    val cd = ClockDomain(myclk,myrst, 
      config = ClockDomainConfig(
      clockEdge        = RISING,
      resetKind        = ASYNC,
      resetActiveLevel = LOW
    ))
    val u_sub0 = cd(new SUB)
}
showRtl(new Top)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.26 15:04:25
[Progress] at 3629.855 : Elaborate components
[Progress] at 3629.857 : Checks and transforms
[Progress] at 3629.860 : Generate Verilog
[Warning] 6 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 3629.863
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 26/10/2019, 15:04:25
// Component : Top


module SUB (
      input  [7:0] a,
      output reg [7:0] b,
      input   myclk,
      input   myrst);
  always @ (posedge myclk or negedge myrst) begin
    if (!myrst) begin
      b <= (8'b00000000);
    end else begin
      b <= a;
    end
  end

endmodule

module Top (
      input   myclk,
      input   myrst);
  wire [7:0] _zz_1_;
  wire [7:0] u_sub0_b;
  SUB u_sub0 ( 
    .a(_zz_1_),
    .b(u_sub0_b),
    .myclk(myclk),
  

defined class Top

In [20]:
class Top extends Component{
    val cd = ClockDomain.external("myClock")
    val u_sub0 = cd(new SUB)
}
showRtl(new Top)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.26 16:19:39
[Progress] at 8143.268 : Elaborate components
[Progress] at 8143.269 : Checks and transforms
[Progress] at 8143.274 : Generate Verilog
[Warning] 6 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 8143.279
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 26/10/2019, 16:19:39
// Component : Top


module SUB (
      input  [7:0] a,
      output reg [7:0] b,
      input   myClock_clk,
      input   myClock_reset);
  always @ (posedge myClock_clk or posedge myClock_reset) begin
    if (myClock_reset) begin
      b <= (8'b00000000);
    end else begin
      b <= a;
    end
  end

endmodule

module Top (
      input   myClock_clk,
      input   myClock_reset);
  wire [7:0] _zz_1_;
  wire [7:0] u_sub0_b;
  SUB u_sub0 ( 
    .a

defined class Top

In [21]:
class ExternalClockExample extends Component {
  val io = new Bundle {
    val result = out UInt (4 bits)
  }

  // On top level you have two signals  :
  //     myClockName_clk and myClockName_reset
  val myClockDomain = ClockDomain.external("myClockName")

  val myArea = new ClockingArea(myClockDomain){
    val myReg = Reg(UInt(4 bits)) init(7)
    myReg := myReg + 1

    io.result := myReg
  }
}
showRtl(new ExternalClockExample)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.26 16:21:00
[Progress] at 8224.146 : Elaborate components
[Progress] at 8224.153 : Checks and transforms
[Progress] at 8224.156 : Generate Verilog
[Done] at 8224.159
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 26/10/2019, 16:21:00
// Component : ExternalClockExample


module ExternalClockExample (
      output [3:0] io_result,
      input   myClockName_clk,
      input   myClockName_reset);
  reg [3:0] myArea_myReg;
  assign io_result = myArea_myReg;
  always @ (posedge myClockName_clk or posedge myClockName_reset) begin
    if (myClockName_reset) begin
      myArea_myReg <= (4'b0111);
    end else begin
      myArea_myReg <= (myArea_myReg + (4'b0001));
    end
  end

endmodule




defined class ExternalClockExample

## 给不同的模块设置默认时钟域

In [ ]:
class MYSub0(cd: ClockDomain) extends Component {
  val io = new Bundle{
    val ai = in UInt(8 bits)
    val ao = out UInt(8 bits)
  }
  io.ao := RegNext(io.ai) init(0)
}

class MYSub1(cd: ClockDomain) extends Component {
  val io = new Bundle{
    val ai = in UInt(8 bits)
    val ao = out UInt(8 bits)
    val a2 = out UInt(8 bits)
  } 
  io.ao := RegNext(io.ai) init(0)
  val cd2 = ClockDomain.external("adc")
  //alow another clockDomain not confict to default clockdomain 
  val area = new ClockingArea(cd2){
    val tmp = RegNext(io.ai) init(0)
    val tmp2 = tmp + (RegNext(io.ai) init(0))
  } 
  io.a2 := area.tmp2
}     

class Top00 extends Component {
  val io = new Bundle{
    val a = in UInt(8 bits)
    val b0 = out UInt(8 bits)
    val b1 = out UInt(8 bits)
    val b2 = out UInt(8 bits)
  }
  val cd0 = ClockDomain.external("cp")
  val cd1 = ClockDomain.external("ap")

  val u_sub0 = cd0(new MYSub0(cd0)) // set u_sub0's default clockDomain with cd0
  val u_sub1 = cd1(new MYSub1(cd1)) // it allow anoter clockDomain in ther module

  u_sub0.io.ai := io.a
  u_sub1.io.ai := io.a

  io.b0 := u_sub0.io.ao
  io.b1 := u_sub1.io.ao
  io.b2 := u_sub1.io.a2

  val tmp = RegNext(io.a) init(0)
}

showRtl(new Top00)

In [ ]:
class CG extends Component{
    val TSE,CLK,E = in Bool
    val ECK = out Bool
    
    val clk_n = !CLK
    val lock_en = Bool.noCombLoopCheck
    when (clk_n) {
        lock_en := E
    }.otherwise {
        lock_en := lock_en
    }
    ECK := lock_en && CLK
}
object CG {
    def apply(cd: ClockDomain, E: Bool, TSE: Bool): Bool = {
        val ret = new CG 
        ret.TSE := TSE
        ret.E := E
        ret.CLK := cd.readClockWire
        ret.ECK
    }
} 
class A extends Component{
    
val clk,rstn,E,TSE = in Bool
val clk_cg = out Bool
val cd =ClockDomain(clk,rstn)
clk_cg := CG(cd,E,TSE)
    
}
showRtl(new A)

In [ ]:
class MySub extends Component {
    val io = new Bundle{
        val a = in UInt(8 bits)
        val b = out UInt(8 bits)
    }
    io.b := RegNext(io.a) init(0)
}
class Top extends Component {
    val io = new Bundle{
        val a0,a1 = in UInt(8 bits)
        val cg_en0,cg_en1 = in Bool
        val b  = out UInt(8 bits)
    }
    
    val u_0 = new CG
    val u_1 = new CG
    u_0.TSE := False
    u_1.TSE := False
    
    u_0.E := io.cg_en0
    u_1.E := io.cg_en1
    
    u_0.CLK := clockDomain.readClockWire
    u_1.CLK := clockDomain.readClockWire 
    
    val gated_clk1: Bool = CG(clockDomain, io.cg_en1, False)
    
    val cgd0 = ClockDomain(u_0.ECK, clockDomain.readResetWire)
    val cgd1 = ClockDomain(u_1.ECK, clockDomain.readResetWire)
    
    val u_sub0 = cgd0(new MySub)
    val u_sub1 = cgd1(new MySub)
    
    u_sub0.io.a := io.a0
    u_sub1.io.a := io.a1
    io.b := RegNext(u_sub0.io.b + u_sub1.io.b) init 0
}

showRtl(new Top)

In [ ]:
class MySub extends Component {
    val io = new Bundle{
        val a = in UInt(8 bits)
        val b = out UInt(8 bits)
    }
    io.b := RegNext(io.a) init(0)
}
class Top extends Component {
    val io = new Bundle{
        val a0,a1 = in UInt(8 bits)
        val cg_en0,cg_en1 = in Bool
        val b  = out UInt(8 bits)
    }
    
    val gated_clk0: Bool = CG(clockDomain, io.cg_en0, False)
    val gated_clk1: Bool = CG(clockDomain, io.cg_en1, False)
    
    val cgd0 = ClockDomain(gated_clk0, clockDomain.readResetWire)
    val cgd1 = ClockDomain(gated_clk1, clockDomain.readResetWire)
    
    val u_sub0 = cgd0(new MySub)
    val u_sub1 = cgd1(new MySub)
    
    u_sub0.io.a := io.a0
    u_sub1.io.a := io.a1
    io.b := RegNext(u_sub0.io.b + u_sub1.io.b) init 0
}

showRtl(new Top)

In [ ]:
val cgd0 = clockDomain.gated(gate_en)
val cgd1 = clockDomain.gatedWithBlackBox(gate_en,io.test_mode)

val cgd2 = AnotherclockDomain.gateWidthBlackBox(gate_en, io.test_mode)

cgd0 cgd1 regard as  the same domain, 
cgd2 is cross domain to cgd0 and cgd1

In [ ]:
class CG extends Component{
    val TSE,CLK,E = in Bool
    val ECK = out Bool
    
    val clk_n = !CLK
    val lock_en = Bool.noCombLoopCheck
    when (clk_n) {
        lock_en := E
    }.otherwise {
        lock_en := lock_en
    }
    ECK := lock_en && CLK
}

object CG {
    def apply(cd: ClockDomain, E: Bool, TSE: Bool): Bool = {
        val ret = new CG 
        ret.TSE := TSE
        ret.E := E
        ret.CLK := cd.readClockWire
        ret.ECK
    }
}

object Utlis{
implicit class  ClockDomainPimper(cd :ClockDomain) {
  def gatedWith(cg_en : Bool, test_mode: Bool = False): ClockDomain = {
    val gated_clk = CG(cd, cg_en, test_mode)
    val ret = ClockDomain(gated_clk, cd.readResetWire)
    ret.setSynchronousWith(cd)
    ret
  }
}
}

import Utlis._


In [ ]:
class Ram_1w_1r(wordWidth: Int, wordCount: Int) extends BlackBox {

  // SpinalHDL will look at Generic classes to get attributes which
  // should be used ad VHDL gererics / Verilog parameter
  // You can use String Int Double Boolean and all SpinalHDL base types
  // as generic value
  val generic = new Generic {
    val wordCount = Ram_1w_1r.this.wordCount
    val wordWidth = Ram_1w_1r.this.wordWidth
  }

  // Define io of the VHDL entiry / Verilog module
  val io = new Bundle {
    val clk = in Bool
    val wr = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(wordCount) bit)
      val data = in Bits (wordWidth bit)
    }
    val rd = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(wordCount) bit)
      val data = out Bits (wordWidth bit)
    }
  }

  //Map the current clock domain to the io.clk pin
  mapClockDomain(clock=io.clk)
}

class TopLevel extends Component {
  val io = new Bundle {
    val wr = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(16) bit)
      val data = in Bits (8 bit)
    }
    val rd = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(16) bit)
      val data = out Bits (8 bit)
    }
  }

  //Instantiate the blackbox
  val ram = new Ram_1w_1r(8,16)

  //Connect all the signals
  io.wr.en   <> ram.io.wr.en
  io.wr.addr <> ram.io.wr.addr
  io.wr.data <> ram.io.wr.data
  io.rd.en   <> ram.io.rd.en
  io.rd.addr <> ram.io.rd.addr
  io.rd.data <> ram.io.rd.data
}
showRtl(new TopLevel)

clken 也可以创建gated效果

In [ ]:
class  T1 extends Component {
    val clk ,rstn,softrstn,clken = in Bool
    val data = in UInt(8 bits)
    val dout = out UInt()
    val coreClockDomain = ClockDomain(clk,rstn,null,softrstn,clken)
    val coreArea = new ClockingArea(coreClockDomain){
    val ret = RegNext(data) init 0
   }
    dout := coreArea.ret
} 

showRtl(new T1)

同一个驱动会被认为是同一时钟域

In [ ]:
class Sub1 extends Component{
    val clk,rstn = in Bool
    val data = in UInt(8 bits)
    val dout = out UInt()
    
   val coreClockDomain = ClockDomain(clk,rstn)
    
   val coreArea = new ClockingArea(coreClockDomain){
     val ret = RegNext(data) init 0
   }
    dout := coreArea.ret
}

In [ ]:
class Top extends Component {
    val clk,rstn = in Bool    
    val data = in UInt(8 bits)
    val dout = out UInt() 
    val u_sub1 = clockDomain(new Sub1)
    val u_sub2 = clockDomain(new Sub1 )
        
        
    u_sub1.clk := clk
    u_sub1.rstn := rstn
    u_sub2.clk := clk
    u_sub2.rstn := rstn
    
    u_sub1.data := data
    u_sub2.data := u_sub1.dout
    
    dout := u_sub2.dout
    
}

showRtl(new Top)

In [ ]:
class T3 extends Component{
    val a = in UInt(6 bits)
    val b = out SInt()
    val c = out Bool
    val remain = SInt
    val tmp = a.resize(7)
    remain := tmp.asSInt - 32
    val ret = 32 - a
    b := remain
    c := remain >= 0
}

showRtl(new T3)

In [ ]:
class T3 extends Component{
    val io = new Bundle{        
    val a = in UInt(6 bits)
    val b = out SInt()        
    val c = out SInt()
    }
    io.b := io.a.asSInt 
    val ret = io.b + 32
    io.c := ret 
}

showRtl(new T3)

In [ ]:
class TopLevel extends Component {
    val a = in UInt(8 bits)
    val b = out UInt(8 bits)
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))
    
  regA := a
  regB := BufferCC(regA, U(0)) 
  b := regB
    
}
showRtl(new TopLevel)

In [ ]:
class TopLevel2 extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))

  val area_clkB = new ClockingArea(clkB){
    val syncA = BufferCC(regA, U(0))

    val tmp = syncA + syncA
    regB := tmp
  }
}
showRtl(new TopLevel2)